In [29]:
import re
import cobra
import ast
import collections
from cobra import Metabolite, Reaction, Model, Gene
import pandas as pd
from cobra.io.dict import model_to_dict, model_from_dict,metabolite_from_dict,gene_from_dict,reaction_from_dict
from cobra.io import read_sbml_model, write_sbml_model

In [30]:
#从（第二步）简化模型——分类文件.xlsx电子表格的'简化代谢物替换'和'简化反应替换'的sheet列表中，将代谢物简化形式重复的代谢物合并，
#同时将反应方程重复的反应合并。随后，整理合并后的代谢物和反应列表，再加上从模型中提取的基因列表，输出excel文件
def meta_comb_rea(data_meta,data_rea,data_gene,path):
    list_meta=[]
    list_rea=[]
    list_gene=[]
    for line in data_meta:
        if line['annotation'] !='已重复':
            dc={}
            dc=collections.OrderedDict()
            dc["id"]=line['CxNy']
            dc["name"]=line['name']
            dc["compartment"]="c"
            dc["charge"]="0"
            dc["formula"]=line['formula']
            dc["annotation"]=line['annotation']
            list_meta.append(dc)
    for line in data_rea:
        if line['annotation'] != '已重复':
            if line['exchange_reac'] == 1:
                if line['代谢物原因而反应为空'] != 1:
                    dc = {}
                    dc = collections.OrderedDict()
                    dc["id"]=line['id']
                    dc["name"]=line['name']
                    dc["lower_bound"]=line['lower_bound']
                    dc["upper_bound"]=line['upper_bound']
                    dc["gene_reaction_rule"]=line['gene_reaction_rule']
                    dc["objective_coefficient"]=line['objective_coefficient']
                    dc["reaction_eq"]=line['reaction_eq']
                    dc["reaction_CNeq"]=line['reaction_CNeq']
                    dc["metabolites"]=line['metabolites']
                    dc["annotation"]=line['annotation']
                    list_rea.append(dc)

    for line in data_gene:
        list_gene.append(line)
    #按照转换sbml程序中对excel文件格式的要求，将STEP.2中输出的excel文件进行整理并输出
    writer = pd.ExcelWriter('（第三步）简化模型——简化模型文件(MC).xlsx')
    pd.DataFrame(list_meta).to_excel(writer, 'metabolites', index=False)
    pd.DataFrame(list_rea).to_excel(writer, 'reactions', index=False)
    pd.DataFrame(list_gene).to_excel(writer, 'genes', index=False)
    writer.save()

In [31]:
#根据从上一步生成的excel文件‘（第三步）简化模型——简化模型文件.xlsx’中提供的代谢物列表、反应列表和基因列表，
#若这些都符合转换的要求（从STEP.2中转换得到的文件即可），则可将表格中的信息转换为sbml文件
def excel2model(path,mname1,mname2):
    model = Model()
    excel = pd.ExcelFile(path+mname1+'.xlsx')
    # 处理代谢物
    mdictl = excel.parse('metabolites', index_col=None).T.to_dict().values()  # 将dataframe转化为list of dictionaries
    for m in mdictl:
        annstr = m['annotation']
        values = re.search(r"OrderedDict\((.*)\)", annstr).group(1)  # 将字符串转换为OrderedDict
        anndict = collections.OrderedDict(ast.literal_eval(values))
        m['annotation'] = anndict
        m['charge'] = int(m['charge'])  # 字符串转化为整数
    model.add_metabolites([metabolite_from_dict(metabolite) for metabolite in mdictl])  # mdictl需要是一个代谢物对象的list而非词典list
    print('add_metabolites OK!')
    # 处理基因
    gdictl = excel.parse('genes', index_col=None).fillna('').T.to_dict().values()
    for g in gdictl:
        annstr = g['annotation']
        if annstr:
            values = re.search(r"OrderedDict\((.*)\)", annstr).group(1)
            anndict = collections.OrderedDict(ast.literal_eval(values))
            g['annotation'] = anndict
    model.genes.extend([gene_from_dict(gene) for gene in gdictl])
    print('genes OK!')
    # 处理反应，因涉及目标反应确定，逐一添加而未从反应list添加
    reactions = excel.parse('reactions', index_col=None).fillna('')  # fillna('')很重要，否则会产生Nan值
    rdictl = reactions.T.to_dict().values()  # 将dataframe转化为listofdictionaries以便模型读取
    for r in rdictl:
#         print(r)
        metstr = r['metabolites']
        values = re.search(r"OrderedDict\((.*)\)", metstr).group(1)
        metdict = collections.OrderedDict(ast.literal_eval(values))
        r['metabolites'] = metdict
        annstr = r['annotation']
        values = re.search(r"OrderedDict\((.*)\)", annstr).group(1)
        anndict = collections.OrderedDict(ast.literal_eval(values))
        r['annotation'] = anndict
        r['lower_bound'] = float(r['lower_bound'])
        r['upper_bound'] = float(r['upper_bound'])
        rea = reaction_from_dict(r, model)  # 由词典得到新反应并添加到模型
        model.add_reaction(rea)
        if r['objective_coefficient']:
            rea.objective_coefficient = float(r['objective_coefficient'])  # 不将字符转化成浮点数也可以，但转化更安全
    model.add_reactions([reaction_from_dict(reaction, model) for reaction in rdictl])
    print('reactions OK!')
    write_sbml_model(model, path+mname2 + "new.xml")

In [32]:
if __name__ == '__main__':
    path = r'/media/jupyter/zhang_rzh/redM(MC)/'
    excel = pd.ExcelFile(path + '（第二步）简化模型——分类文件(MC).xlsx')
    data_meta = excel.parse('简化代谢物替换', index_col=None).T.to_dict().values()
    data_rea = excel.parse('简化反应替换', index_col=None).T.to_dict().values()
    #读取STEP.2中输出文件，从'简化代谢物替换'和'简化反应替换'的sheet中提取简化分类后的反应列表和代谢物列表
    excel_gene = pd.ExcelFile('metacyc18620new.xlsx')
    data_gene = excel_gene.parse('genes', index_col=None).T.to_dict().values()
    #从模型中提取基因列表
    mname = 'metacyc-18620'#给输出简化模型命名
    meta_comb_rea(data_meta,data_rea,data_gene,path)#整理代谢物、反应和基因的信息并输出Excel文件
    print('---------------------------------------------------------------------------------------------------------------')
    excel2model(path,'（第三步）简化模型——简化模型文件(MC)',mname)#根据上一步输出的excel文件转换为sbml文件
    print('finish!!')

---------------------------------------------------------------------------------------------------------------
add_metabolites OK!
genes OK!


Ignoring reaction 'RXN__45__10638__45__1__45__183__45__2__45__183__45__SN__45__GLYCEROL__45__PHOSPHOCHOLINE__47__SER__47____47__CPD__45__8342__47__CHOLINE__46__61__46__' since it already exists.
Ignoring reaction 'PHOSCHOL__45__RXN__45__1__45__183__45__2__45__183__45__SN__45__GLYCEROL__45__PHOSPHOCHOLINE__47__WATER__47____47__CPD__45__8274__47__CHOLINE__47__PROTON__46__70__46__' since it already exists.
Ignoring reaction 'RXN__45__1641__45__1__45__2__45__DIPALMITOYLPHOSPHATYLCHOLINE__47__CPD__45__17271__47____47__CPD66__45__35__47__CPD__45__8345__46__64__46__' since it already exists.
Ignoring reaction 'RXN-1641-1-2-DIPALMITOYLPHOSPHATIDYLCHOLINE/CPD66-34//CPD66-35/CPD-8343.63.' since it already exists.
Ignoring reaction 'RXN-10638-1-2-DIPALMITOYLPHOSPHATIDYLCHOLINE/SER//CPD-12817/CHOLINE.58.' since it already exists.
Ignoring reaction 'RXN-15065' since it already exists.
Ignoring reaction 'PHOSCHOL-RXN-1-2-DIPALMITOYLPHOSPHATIDYLCHOLINE/WATER//CPD0-1422/CHOLINE/PROTON.67.' since it al

Ignoring reaction '_4__46__1__46__2__46__20__45__RXN' since it already exists.
Ignoring reaction '2KETO-3METHYLVALERATE-RXN' since it already exists.
Ignoring reaction '2.6.1.32-RXN' since it already exists.
Ignoring reaction '2.6.1.14-RXN-2-KETO-6-AMINO-CAPROATE/ASN//2-OXOSUCCINAMATE/LYS.50.' since it already exists.
Ignoring reaction 'RXN-8243' since it already exists.
Ignoring reaction '2.6.1.14-RXN-2-KETO-GLUTARAMATE/ASN//2-OXOSUCCINAMATE/GLN.45.' since it already exists.
Ignoring reaction 'RXN-13072' since it already exists.
Ignoring reaction '2-ISOPROPYLMALATESYN-RXN' since it already exists.
Ignoring reaction '1.2.1.25-RXN' since it already exists.
Ignoring reaction '2.6.1.14-RXN-2-KETOGLUTARATE/ASN//2-OXOSUCCINAMATE/GLT.42.' since it already exists.
Ignoring reaction 'RXN__45__14123' since it already exists.
Ignoring reaction 'RXN__45__17554' since it already exists.
Ignoring reaction '2.6.1.22-RXN' since it already exists.
Ignoring reaction '_2__46__6__46__1__46__33__45__RXN' 

Ignoring reaction 'RXN-8032' since it already exists.
Ignoring reaction 'PSERTRANSAM-RXN' since it already exists.
Ignoring reaction 'RXN__45__18395' since it already exists.
Ignoring reaction 'R241__45__RXN' since it already exists.
Ignoring reaction 'RXN-11210' since it already exists.
Ignoring reaction 'BETA-UREIDOPROPIONASE-RXN' since it already exists.
Ignoring reaction '_2__46__3__46__1__46__114__45__RXN' since it already exists.
Ignoring reaction 'RXN__45__15527' since it already exists.
Ignoring reaction 'RXN0__45__5065' since it already exists.
Ignoring reaction 'PSERTRANSAMPYR-RXN' since it already exists.
Ignoring reaction 'RXN-7929' since it already exists.
Ignoring reaction 'ADCLY-RXN' since it already exists.
Ignoring reaction 'R62__45__RXN' since it already exists.
Ignoring reaction 'GABATRANSAM-RXN' since it already exists.
Ignoring reaction 'RXN-13328' since it already exists.
Ignoring reaction 'RXN__45__11222' since it already exists.
Ignoring reaction 'RXN-6902' sinc

Ignoring reaction 'RXN__45__18669' since it already exists.
Ignoring reaction 'RXN-0' since it already exists.
Ignoring reaction 'RXN-7743' since it already exists.
Ignoring reaction 'RXN3DJ__45__35528' since it already exists.
Ignoring reaction 'RXN0-7165' since it already exists.
Ignoring reaction 'SPERMACTRAN__45__RXN' since it already exists.
Ignoring reaction 'RXN__45__9939' since it already exists.
Ignoring reaction 'RXN-16560' since it already exists.
Ignoring reaction 'MALSYN-RXN' since it already exists.
Ignoring reaction 'ACETYL-COA-ACETYLTRANSFER-RXN' since it already exists.
Ignoring reaction 'TEICHOICSYN3__45__RXN' since it already exists.
Ignoring reaction '_3__46__2__46__2__46__11__45__RXN' since it already exists.
Ignoring reaction '_3__46__5__46__1__46__26__45__RXN' since it already exists.
Ignoring reaction 'RXN__45__15245' since it already exists.
Ignoring reaction 'RXN__45__15247' since it already exists.
Ignoring reaction 'RXN-12726' since it already exists.
Ignori

...

Ignoring reaction 'RXN__45__18419' since it already exists.
Ignoring reaction 'RXN__45__18564' since it already exists.
Ignoring reaction 'RXN__45__19093' since it already exists.
Ignoring reaction '_2__46__1__46__1__46__132__45__RXN' since it already exists.
Ignoring reaction 'RXN__45__12805' since it already exists.
Ignoring reaction 'RXN__45__18224' since it already exists.
Ignoring reaction 'RXN__45__18516' since it already exists.
Ignoring reaction 'RXN__45__18517' since it already exists.
Ignoring reaction 'RXN__45__18519' since it already exists.
Ignoring reaction 'RXN-13404' since it already exists.
Ignoring reaction 'RXN__45__18337' since it already exists.
Ignoring reaction 'RXN-13405' since it already exists.
Ignoring reaction 'RXN__45__13752' since it already exists.
Ignoring reaction 'RXN__45__13403' since it already exists.
Ignoring reaction 'RXN__45__11105' since it already exists.
Ignoring reaction 'RXN__45__11261' since it already exists.
Ignoring reaction 'RXN__45__11

Ignoring reaction 'RXN-16562' since it already exists.
Ignoring reaction 'RXN__45__18551' since it already exists.
Ignoring reaction 'THIAMIN__45__PYRINYLASE__45__RXN' since it already exists.
Ignoring reaction 'RXN__45__11103' since it already exists.
Ignoring reaction 'RXN__45__9377' since it already exists.
Ignoring reaction 'RXN__45__14569' since it already exists.
Ignoring reaction '_4__46__3__46__1__46__16__45__RXN' since it already exists.
Ignoring reaction 'THI-P-SYN-RXN' since it already exists.
Ignoring reaction 'RXN__45__863' since it already exists.
Ignoring reaction 'RXN__45__15950' since it already exists.
Ignoring reaction 'RXN__45__8254' since it already exists.
Ignoring reaction 'RXN__45__15667' since it already exists.
Ignoring reaction 'RXN__45__19475' since it already exists.
Ignoring reaction 'RXN__45__17904' since it already exists.
Ignoring reaction 'RXN__45__16124' since it already exists.
Ignoring reaction 'RXN-15578' since it already exists.
Ignoring reaction 

Ignoring reaction 'UROGENDECARBOX__45__RXN' since it already exists.
Ignoring reaction '_2__46__7__46__7__46__64__45__RXN__45__UTP__47__CPD__45__16502__47__PROTON__47____47__UDP__45__SULFOQUINOVOSE__47__PPI__46__45__46__' since it already exists.
Ignoring reaction 'RXN__45__13760' since it already exists.
Ignoring reaction 'RXN__45__17004' since it already exists.
Ignoring reaction 'RXN__45__19194' since it already exists.
Ignoring reaction 'RXN__45__14320' since it already exists.
Ignoring reaction 'RXN-2-VANILLATE/CPD-1301//3-4-DIHYDROXYBENZOATE/CPD-1302.51.' since it already exists.
Ignoring reaction 'RXN__45__10885' since it already exists.
Ignoring reaction 'RXN-10891' since it already exists.
Ignoring reaction 'RXN-2-VANILLATE/THF//3-4-DIHYDROXYBENZOATE/5-METHYL-THF.50.' since it already exists.
Ignoring reaction 'RXN__45__8201' since it already exists.
Ignoring reaction 'RXN__45__12087' since it already exists.
Ignoring reaction 'RXN__45__9705' since it already exists.
Ignoring 

Ignoring reaction 'RXN__45__9940' since it already exists.
Ignoring reaction 'ACECOATRANS-RXN-PALMITYL-COA/ACET//PALMITATE/ACETYL-COA.40.' since it already exists.
Ignoring reaction 'RXN__45__20564' since it already exists.
Ignoring reaction 'AMINEPHEN__45__RXN__91__CCO__45__PERI__45__BAC__93____45__PHENYLETHYLAMINE__47__OXYGEN__45__MOLECULE__47__WATER__47____47__PHENYLACETALDEHYDE__47__AMMONIUM__47__HYDROGEN__45__PEROXE__46__100__46__' since it already exists.
Ignoring reaction 'GUANINE-DEAMINASE-RXN' since it already exists.
Ignoring reaction 'ACETYLCHOLINESTERASE-RXN' since it already exists.


reactions OK!
finish!!
